The intent of this script is to create a series of summary statistics to better understand the ECDS data and the distrubtion of diagnostic procedures.
note: some of the sensitive (not sure if I can share) part of the code is removed and replaced witha  comment

In [ ]:

import pandas as pd
import pyodbc
import numpy as np
import csv
import matplotlib.pyplot as plt
from scipy.stats.contingency import association
import dash
from dash import html
from dash import dcc
import numpy as np
import pandas as pd
import dash_cytoscape as cyto
import networkx as nx

In [ ]:
conn = pyodbc.connect() #connection details for SUS database

In [ ]:
df = pd.read_sql_query() #simple select querry to pull from data table


In [ ]:
#create counts for investigation codes
invs_count = df['Der_Number_EC_Investigation'].value_counts(sort=False)
Prov_count = df['Provider_Code'].value_counts(sort=True)

sinvs = invs_count.sort_index()
sinvs1 = sinvs.drop(sinvs.index[0])
#create some graphs looking at the distribution of investigation count
fig, ax = plt.subplots(2)
fig.suptitle("The Number of Unique Investigation Conducted")
ax[0].plot(sinvs)
ax[0].set_title("Single Investigation included")
ax[0].xaxis.set_ticks(np.arange(1,25,1))
ax[1].plot(sinvs1)
ax[1].xaxis.set_ticks(np.arange(1,25,1))
ax[1].set_title("Single Investigations excluded")
fig.tight_layout()


In [ ]:
##create counts per provider and exclude patients with just 1 investigation
inv_prov_count = df.groupby(['Provider_Code','Der_Number_EC_Investigation']).size().to_frame('size').reset_index()
inv_prov_count = inv_prov_count[inv_prov_count.Der_Number_EC_Investigation != 1]
fig, ax = plt.subplots()
for key, grp in inv_prov_count.groupby(['Provider_Code']):
    ax = grp.plot(ax= ax, kind='line', x= 'Der_Number_EC_Investigation', y= 'size')


plt.legend().remove()
plt.show()

inv_prov_count.groupby(['Provider_Code']).mean()
inv_prov_count

In [ ]:
#Look at the Mean/Max/min per provider of number of investigation done. 
means_ivc = inv_prov_count.groupby(['Provider_Code']).mean().reset_index()
max_ivc = inv_prov_count.groupby(['Provider_Code']).max().reset_index()
min_means = means_ivc.min()
max_means = means_ivc.max()
print(min_means, max_means)
fig, ax = plt.subplots()
#fig(num=None, figsize=(15,5), dpi=300, facecolor='w', edgecolor='r')
ax.plot(means_ivc.Provider_Code, means_ivc.Der_Number_EC_Investigation)
fig.set_figwidth(10)
ax.xaxis.set_visible(False)

fig, ax = plt.subplots()
fig.suptitle("Break down of Mean-tests per provider")
ax.hist(means_ivc.Der_Number_EC_Investigation, bins=13, linewidth=0.4, edgecolor="white")
ax.set_xlabel("Mean Tests")
fig.set_figwidth(10)

ax.set(xlim=(0, 13), 
       ylim=(0, 32), )

In [ ]:
df_test = pd.read_sql_query() #querry that creates distinct counts of the provider per investigation - not sure why I did this way


In [ ]:
#looking at breakdown per investigation code and histogram
investigation_Count = df_test.groupby('InvestigationDescription').size().to_frame('Count').reset_index()
investigation_Count.sort_values(by= 'Count', ascending=True)
fig, ax = plt.subplots()
ax.barh(y=investigation_Count.InvestigationDescription, width= sorted(investigation_Count.Count), align='center')
fig.set_figheight(10)
fig, ax = plt.subplots()
ax.hist(investigation_Count.Count, bins=20, linewidth=0.4, edgecolor="white")

In [ ]:
#Looking at hte cross tab between provider and investigation to see if there is any provider the like one more than the other. 
inv_prov_test_corr =  pd.read_sql_query() # Looking at just the provider and investigation only
pd.crosstab(inv_prov_test_corr.Provider_Code, inv_prov_test_corr.InvestigationDescription)
ot = pd.crosstab(inv_prov_test_corr.Provider_Code, inv_prov_test_corr.InvestigationDescription)

matric_corss = association(pivot)
print(matric_corss)

In [ ]:
piv

In [ ]:
inv_correlations = pd.read_sql_query() # Looking purely patient and investigation ow

In [ ]:
#Creates a cross tab of investigation by investigation depending on how many times they have a similar patient. This can be the start of the network analysis(But I do not use it)
pivot = pd.crosstab(inv_correlations.EC_Ident, inv_correlations.InvestigationDescription)
c = pivot.T @ pivot
c = c.mask(np.triu(c, 1) == 0)
c = c.stack()
c = c.rename_axis(['name1', 'name2']).reset_index(name='count')


In [ ]:
c.head()